<a href="https://colab.research.google.com/github/Hongzf/OLID/blob/main/Machine_Learning_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install wordsegment
!pip install transformers
!pip install trainer
!pip install attention

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython.display import HTML
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import os
import emoji
from wordsegment import load, segment
import pickle
import torch
from torch.utils.data import Dataset

import os,re
from bs4 import BeautifulSoup
from gensim import models

from transformers import BertTokenizer, RobertaTokenizer, get_cosine_schedule_with_warmup
from trainer import Trainer 
from torch import nn
from transformers import BertModel, BertForSequenceClassification, RobertaForSequenceClassification, RobertaModel
from attention import Attention
load()

In [ ]:
#optional Google drive integration
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OLID_PATH = './drive/MyDrive/OLID'
SAVE_PATH = '.drive/MyDrive/OLID/save'
LABEL_DICT = {
    'a': {'OFF': 0, 'NOT': 1},
    'b': {'TIN': 0, 'UNT': 1, 'NULL': 2},
    'c': {'IND': 0, 'GRP': 1, 'OTH': 2, 'NULL': 3}
}

TRAIN_PATH = os.path.join(OLID_PATH, 'olid-training-v1.0.tsv')

In [ ]:
task = 'a'
# model_name = 'roberta'
# model_size = 'base'
truncate = 512
epochs = 10
lr = 0.0001
wd = 0.0
bs = 50 #72
patience = 5

Preprocessing

In [ ]:
def emoji2word(sents):
    return [emoji.demojize(sent) for sent in sents]

def remove_useless_punctuation(sents):
    for i, sent in enumerate(sents):
        sent = sent.replace(':', ' ')
        sent = sent.replace('_', ' ')
        sent = sent.replace('...', ' ')
        sents[i] = sent
    return sents

def remove_replicates(sents):
    # if there are multiple `@USER` tokens in a tweet, replace it with `@USERS`
    # because some tweets contain so many `@USER` which may cause redundant
    for i, sent in enumerate(sents):
        if sent.find('@USER') != sent.rfind('@USER'):
            sents[i] = sent.replace('@USER', '')
            sents[i] = '@USERS ' + sents[i]
    return sents

def replace_rare_words(sents):
    rare_words = {
        'URL': 'http'
    }
    for i, sent in enumerate(sents):
        for w in rare_words.keys():
            sents[i] = sent.replace(w, rare_words[w])
    return sents

def segment_hashtag(sents):
    # E.g. '#LunaticLeft' => 'lunatic left'
    for i, sent in enumerate(sents):
        sent_tokens = sent.split(' ')
        for j, t in enumerate(sent_tokens):
            if t.find('#') == 0:
                sent_tokens[j] = ' '.join(segment(t))
        sents[i] = ' '.join(sent_tokens)
    return sents


def process_tweets(tweets):
    tweets = emoji2word(tweets)
    tweets = replace_rare_words(tweets)
    tweets = remove_replicates(tweets)
    tweets = segment_hashtag(tweets)
    tweets = remove_useless_punctuation(tweets)
    tweets = np.array(tweets)
    return tweets

In [ ]:
df = pd.read_csv(TRAIN_PATH, sep='\t', keep_default_na=False)

if task == 'a':
        df = df
elif task == 'b':
        df=df[~df['subtask_b'].isin(['NULL'])]
elif task == 'c':
        df=df[~df['subtask_c'].isin(['NULL'])]

ids = np.array(df['id'].values)
tweets = np.array(df['tweet'].values)

print(tweets[1])
# Process tweets
df['tweet'] = process_tweets(tweets)

tweets_2 = df['tweet']
print(tweets_2[1])

label_a = np.array(df['subtask_a'].values)
label_b = df['subtask_b'].values
label_c = np.array(df['subtask_c'].values)
nums = len(df)

if task == 'a':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_b','subtask_c'],axis = 1)
elif task == 'b':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_a','subtask_c'],axis = 1)
        df_train=df_train[~df_train['subtask_b'].isin(['NULL'])]
elif task == 'c':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_a','subtask_b'],axis = 1)
        df_train=df_train[~df_train['subtask_c'].isin(['NULL'])]



# read_test_file(task, tokenizer, truncate=512):
df1 = pd.read_csv(os.path.join(OLID_PATH, 'testset-level' + task + '.tsv'), sep='\t')
df2 = pd.read_csv(os.path.join(OLID_PATH, 'labels-level' + task + '.csv'), sep=',')
ids = np.array(df1['id'].values)
# tweets = np.array(df1['tweet'].values)
# labels = np.array(df2['label'].values)
# nums = len(df1)
df_test = pd.merge(df1,df2,on='id')
# Process tweets
tweets_test = df_test['tweet']
df_test['tweet'] = process_tweets(tweets_test)

#     # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     token_ids = [tokenizer(text=tweets[i], add_special_tokens=True, max_length=truncate) for i in range(nums)]
#     mask = np.array(get_mask(token_ids))
#     lens = get_lens(token_ids)
#     token_ids = np.array(pad_sents(token_ids, tokenizer.pad_token_id))

#     return ids, token_ids, lens, mask, labels

In [ ]:
print(emoji.demojize('Python is 👍'))

In [ ]:
print(tweets[:20])

In [ ]:
print(tweets_2)

In [ ]:
print(df)

In [ ]:
print(df_train)

In [ ]:
print(df_test)

tokenizer (TF-IDF)

In [ ]:
# TF-IDF feature extract
# Create the numericalizer TFIDF for lowercase
tfidf_vec  = TfidfVectorizer(decode_error='ignore', lowercase=True, stop_words="english")

# # translate the input text data
# # Numericalize the train combined sentence
X_train = tweets_2
X_val = tweets_test
Y_train_name = 'subtask_'+ task
y_train = df_train[Y_train_name]
y_val = df_test['label']
x_train_tfidf_matrix = tfidf_vec.fit_transform(X_train.astype('U'))
# Numericalize the test Headline
x_valid_tfidf_matrix = tfidf_vec.transform(X_val.astype('U'))

In [ ]:
y_train

In [ ]:
x_train_tfidf_matrix

Brief Visualization

In [ ]:
print('Train size: ', x_train_tfidf_matrix.shape)
print('Val size: ', x_valid_tfidf_matrix.shape)

In [ ]:
dictionary = np.asarray(tfidf_vec.get_feature_names())
print(dictionary[np.random.randint(0,len(dictionary),size=20)])

In [ ]:
df['tweet']

Model

SVM

In [ ]:
from sklearn.svm import LinearSVC, SVC
# load the machine model form sklearn
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, naive_bayes, svm

# Define the parameters to tune
parameter_grid = { 
    'C': [1.0, 10],
    'gamma': [1, 'auto', 'scale']
}
# # setup the SVM model
SVM_model = GridSearchCV(svm.SVC(kernel='linear'), parameter_grid, cv=5, n_jobs=-1)
# SVM_model = LinearSVC(C=1, loss="hinge")
# SVM_model = SVC(kernel="linear")
# fit the model with training dataset
# SVM_model.fit(x_train_tfidf_matrix, y_train)

SVM_model.fit(x_train_tfidf_matrix, y_train)

In [ ]:
grid_search = GridSearchCV(SVM_model, parameter_grid, cv=5,
                           n_jobs =-1,
                           scoring='accuracy',
                           return_train_score=True)

grid_search.fit(x_train_tfidf_matrix, y_train)
print(grid_search.best_params_)

In [ ]:
predictions_SVM = SVM_model.predict(x_valid_tfidf_matrix)

In [ ]:
len(predictions_SVM)

860

In [ ]:
# on test dataset
cm = metrics.confusion_matrix(y_val, predictions_SVM)

print('Confusion matrix：', cm, sep='\n')

TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# print(FP, FN, TP, TN)

# accuracy: (TP + TN) / (TP + TN + FP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy: {:.3}%".format(accuracy*100))

# sensitivity(recall rate): TP / (TP + FN)
sensitivity = TP / (TP + FN)
print("Sensitivity: {:.3}%".format(sensitivity*100))

# specificity: TN / (FP + TN)
specificity = TN / (FP + TN)
print("Specificity: {:.3}%".format(specificity*100))

# precision: TP / (TP + FP)
precision = TP / (TP + FP)
print("Precision: {:.3}%".format(precision*100))

# F1 score: 2 * (precision * sensitivity) / (precision + sensitivity)
print("F1 score: {:.3}%".format(2 * (precision * sensitivity) / (precision + sensitivity)*100))

# classification report
r = metrics.classification_report(y_val, predictions_SVM)
print('Classification report：', r, sep='\n')

Random forest

In [ ]:
RF = RandomForestClassifier()
RF.fit(x_train_tfidf_matrix, y_train)
# show model


RandomForestClassifier()

In [ ]:
# get the train score from the train set
print("SVM Training Score -> {}%".format(RF.score(x_train_tfidf_matrix, y_train)*100))
# predict the result labels on validation dataset
predictions_RF = RF.predict(x_valid_tfidf_matrix)
# predict vith the valid data set
print("SVM Accuracy Score -> {}%".format(RF.score(x_valid_tfidf_matrix, y_val)*100))

SVM Training Score -> 99.89680082559339%
SVM Accuracy Score -> 62.91079812206573%


In [ ]:
# on test dataset
cm = metrics.confusion_matrix(y_val, predictions_RF)

print('Confusion matrix：', cm, sep='\n')

TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# print(FP, FN, TP, TN)

# accuracy: (TP + TN) / (TP + TN + FP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy: {:.3}%".format(accuracy*100))

# sensitivity(recall rate): TP / (TP + FN)
sensitivity = TP / (TP + FN)
print("Sensitivity: {:.3}%".format(sensitivity*100))

# specificity: TN / (FP + TN)
specificity = TN / (FP + TN)
print("Specificity: {:.3}%".format(specificity*100))

# precision: TP / (TP + FP)
precision = TP / (TP + FP)
print("Precision: {:.3}%".format(precision*100))

# F1 score: 2 * (precision * sensitivity) / (precision + sensitivity)
print("F1 score: {:.3}%".format(2 * (precision * sensitivity) / (precision + sensitivity)*100))

# classification report
r = metrics.classification_report(y_val, predictions_RF)
print('Classification report：', r, sep='\n')

In [ ]:
param_grid = [
    {'n_estimators': [5, 20, 40, 60], 'max_features': [2, 12, 24, 48]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

grid_search = GridSearchCV(RF, param_grid, cv=5,
                           scoring='accuracy',
                           return_train_score=True)

grid_search.fit(x_train_tfidf_matrix, y_train)

In [ ]:
print(grid_search.best_params_)

{'max_features': 28, 'n_estimators': 30}


In [ ]:
RF_new = RandomForestClassifier(n_estimators = 30,max_features = 30).fit(x_train_tfidf_matrix, y_train)

predictions_RF_new = RF_new.predict(x_valid_tfidf_matrix)
# on test dataset
cm = metrics.confusion_matrix(y_val, predictions_RF_new)

print('Confusion matrix：', cm, sep='\n')

TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# print(FP, FN, TP, TN)

# accuracy: (TP + TN) / (TP + TN + FP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy: {:.3}%".format(accuracy*100))

# sensitivity(recall rate): TP / (TP + FN)
sensitivity = TP / (TP + FN)
print("Sensitivity: {:.3}%".format(sensitivity*100))

# specificity: TN / (FP + TN)
specificity = TN / (FP + TN)
print("Specificity: {:.3}%".format(specificity*100))

# precision: TP / (TP + FP)
precision = TP / (TP + FP)
print("Precision: {:.3}%".format(precision*100))

# F1 score: 2 * (precision * sensitivity) / (precision + sensitivity)
print("F1 score: {:.3}%".format(2 * (precision * sensitivity) / (precision + sensitivity)*100))

# classification report
r = metrics.classification_report(y_val, predictions_RF_new)
print('Classification report：', r, sep='\n')

Logistic regression

In [ ]:
LR = LogisticRegression()
LR.fit(x_train_tfidf_matrix, y_train)
# show model


LogisticRegression()

In [ ]:
# get the train score from the train set
print("SVM Training Score -> {}%".format(LR.score(x_train_tfidf_matrix, y_train)*100))
# predict the result labels on validation dataset
predictions_LR = LR.predict(x_valid_tfidf_matrix)
# predict vith the valid data set
print("SVM Accuracy Score -> {}%".format(LR.score(x_valid_tfidf_matrix, y_val)*100))

SVM Training Score -> 81.99174406604747%
SVM Accuracy Score -> 64.7887323943662%


In [ ]:
# on test dataset
cm = metrics.confusion_matrix(y_val, predictions_LR)

print('Confusion matrix：', cm, sep='\n')

TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# print(FP, FN, TP, TN)

# accuracy: (TP + TN) / (TP + TN + FP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy: {:.3}%".format(accuracy*100))

# sensitivity(recall rate): TP / (TP + FN)
sensitivity = TP / (TP + FN)
print("Sensitivity: {:.3}%".format(sensitivity*100))

# specificity: TN / (FP + TN)
specificity = TN / (FP + TN)
print("Specificity: {:.3}%".format(specificity*100))

# precision: TP / (TP + FP)
precision = TP / (TP + FP)
print("Precision: {:.3}%".format(precision*100))

# F1 score: 2 * (precision * sensitivity) / (precision + sensitivity)
print("F1 score: {:.3}%".format(2 * (precision * sensitivity) / (precision + sensitivity)*100))

# classification report
r = metrics.classification_report(y_val, predictions_LR)
print('Classification report：', r, sep='\n')

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)
GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [ ]:
LRparam_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 3, cv=5)

# fitting the model for grid search 
LR_search.fit(x_train_tfidf_matrix, y_train)
LR_search.best_params_
# summarize
print('Mean Accuracy: %.3f' % LR_search.best_score_)
print('Config: %s' % LR_search.best_params_)

In [ ]:
LR_search.best_params_

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)